In [1]:
import time
start_time = time.time()
import cv2
import numpy as np
import math
from PIL import Image
%matplotlib auto
image = cv2.imread('bird.jpg',1)
cv2.imshow('original', image)

Using matplotlib backend: Qt5Agg


In [2]:
img_str = cv2.imencode('.jpg', image)[1].tobytes()
if len(img_str)%8 != 0:#check for compatibility
    while len(img_str)%8 != 0:
        img_str += b'\x00'

og_img_np = np.frombuffer(img_str, dtype=np.uint8)

In [3]:
import blowfish
cipher = blowfish.Cipher(b'passwordisnotapassword')

In [4]:
data = img_str # electronic codebook mode

data_encrypted = b"".join(cipher.encrypt_ecb(data))

cipher1 = blowfish.Cipher(b'passwordisnotapass')
data_encrypted1 = b"".join(cipher1.encrypt_ecb(data_encrypted))
data_decrypted1 = b"".join(cipher1.decrypt_ecb(data_encrypted1))

data_decrypted = b"".join(cipher.decrypt_ecb(data_decrypted1))

#assert data == data_decrypted

In [5]:
img_str_np = np.frombuffer(data_decrypted, dtype=np.uint8)

In [6]:
img = cv2.imdecode(img_str_np, 1)

In [7]:
cv2.imshow('decrypted', img)

In [8]:
#img_str2 = cv2.imencode('.jpg', img)[1].tobytes()
#img_str2 = img_str2[0:len(img_str)]
#img_str_np2 = np.frombuffer(img_str2, dtype=np.uint8)

In [9]:
img_str_np_enc = np.frombuffer(data_encrypted, dtype=np.uint8)
a = len(img_str_np_enc)
i = 1
while a%2 == 0:
    i = i*2
    a = a//2


img_str_np_enc1 = img_str_np_enc.reshape(i,len(img_str_np_enc)//i)

In [10]:
enc_img = Image.fromarray(img_str_np_enc1)


In [11]:
enc_img.show()

In [12]:
def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(255 / math.sqrt(mse))

d = psnr(image, img)
print(d)

43.22586472210333


In [13]:
cv2.PSNR(image, img)

42.722213649793986

In [14]:
MSE = np.square(np.subtract(img_str_np,og_img_np)).mean()
MSE

0.0

In [15]:
Y = np.square(np.subtract(image,img)).mean()
print("MSE:", Y)

MSE: 3.0938117793787896


In [16]:
np.corrcoef(image.ravel(), img.ravel())

array([[1.        , 0.99943421],
       [0.99943421, 1.        ]])

In [17]:
def entropy(array):
    total_entropy = 0
    for i in array:
        total_entropy += -i * math.log(i, 2)
    return total_entropy

In [18]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 6.07799220085144 seconds ---


In [19]:
entropy(img.ravel())

ValueError: math domain error